# 03 - Evaluation, Model-Gating & Smoke Tests
Purpose:
 - evaluate the model and compute gating decision
 - provide smoke test code that will be used by CI to validate a deployed API

This notebook contains:
 - metric computation used for gating (example: val_accuracy threshold)
 - example of hitting the running FastAPI service and asserting a valid response


In [ ]:
from pathlib import Path
import json
import joblib
import logging
import requests
from typing import Dict

ROOT = Path.cwd()
MODEL_DIR = ROOT / "model_registry"
logging.basicConfig(level=logging.INFO)


## 3.1 — Compute gating decision locally
Replace threshold rules with your production gating policy.


In [ ]:
def compute_gating(meta_path: str, threshold: float=0.70) -> Dict:
    """
    Read model metadata and compute whether the model passes gating rules.
    Returns a dict with pass/fail and reason(s).
    """
    meta = json.loads(Path(meta_path).read_text())
    acc = float(meta.get("val_accuracy", 0.0))
    passed = acc >= threshold
    return {"passed": passed, "val_accuracy": acc, "threshold": threshold, "meta": meta}

# example: pick latest meta
meta_files = list(MODEL_DIR.glob("*.meta.json"))
meta_files.sort(key=lambda p: p.stat().st_mtime, reverse=True)
if not meta_files:
    raise SystemExit("No model metadata files found. Run training notebook first.")
result = compute_gating(str(meta_files[0]), threshold=0.70)
result


## 3.2 — Smoke test for deployed service
This is a CI-friendly smoke test that:
 - posts a sample payload to /predict
 - asserts 200 and valid JSON with `predictions` key
This code can be used as pytest test or as a GitHub Actions step.


In [ ]:
def smoke_test_predict(endpoint: str = "http://localhost:8000/predict", sample_payload=None, timeout=5):
    """
    Run a basic smoke test against inference API.
    Returns: dict(result, status_code, body)
    """
    if sample_payload is None:
        sample_payload = {"features": [[5.1, 3.5, 1.4, 0.2]]}
    try:
        r = requests.post(endpoint, json=sample_payload, timeout=timeout)
        r.raise_for_status()
        body = r.json()
        ok = isinstance(body, dict) and "predictions" in body
        return {"ok": ok, "status_code": r.status_code, "body": body}
    except Exception as exc:
        return {"ok": False, "error": str(exc)}

# Example run (CI will point endpoint to deployed service)
print(smoke_test_predict("http://127.0.0.1:8000/predict"))


## 3.3 — Example: integrate gating & smoke test in CI
Pseudocode for CI job:

 - Run training job -> produces model artifact and meta
 - compute_gating(meta) -> if pass then deploy canary
 - after canary deployed, run smoke_test_predict against canary endpoint
 - if smoke ok and metrics (from prometheus) pass, promote; else rollback

End of notebook 03.
